# Intro
오늘도 한가로운 심심한 나🫢

인터넷에서 방황하다 [EdNet](https://github.com/riiid/ednet) 데이터 발견🤩

우와! Education 데이터다

(호기심 가득 가득) 오늘 미팅도 없는데 딴짓이나 해볼까?

요즘 유행(?)한다는 *데이터 드리이이이븐 프로덕트*를 만들어봐야지!! 

이름부터 정해보자 음... [산타(Santa)](https://www.youtube.com/c/SANTA990)을 넘어설 **루돌프(Rudolph)**로 하겠어

일단, 데이터부터 불러와야지

# 1편 데이터 불러오기

### DataSet
- example_sample_submission.csv(971 B)
- example_test.csv(6.14 KiB)
- lectures.csv(9.7 KiB)
- questions.csv(296.16 KiB)
- train.csv(5.85 GiB)

데이터를 불러오자!!! 슝슝슝~

어라~ 왜 이렇게 커?!?! 커도 너무 큰데

전처리 하나 하는 순간 Ram이 터질 것만 같은 느낌?!

오기가(5GiB)를 보니 오기가 생긴다 🧐

용량 줄이는 법을 어디선가 본 기억이 (뒤적뒤적) 찾았다!! [여기 링크](https://www.kaggle.com/code/rohanrao/tutorial-on-reading-large-datasets) 

역시 캐글에는 너무 친절한 분들이 많아 🙏🙏🙏

대충 읽어보니 4가지나 있네. 난 판다스 밖에 모르니깐 무조건 판다스 Pick

In [ ]:
# %%time
# import pandas as pd

# dtypes = {
#     "row_id": "int64",
#     "timestamp": "int64",
#     "user_id": "int32",
#     "content_id": "int16",
#     "content_type_id": "boolean",
#     "task_container_id": "int16",
#     "user_answer": "int8",
#     "answered_correctly": "int8",
#     "prior_question_elapsed_time": "float32", 
#     "prior_question_had_explanation": "boolean"
# }

# train = pd.read_csv("../input/riiid-test-answer-prediction/train.csv", dtype=dtypes)
# print("Train size:", train.shape)

데이터는 5분 뒤 공개합니다 ㅠㅠ

이럴 수가!!! 데이터 로드에 5분?? 이건 아니야 ~

더 좋은 방법이 있다고? 

**멋진 동료**: 데이터 셋을 (피클 형태로 저장해서 그 피클을 불러오면) 20초 만에 불러올 수 있습니다.

20초 만에?? 대에박~ 빨리 알려주세요. 커피 쏠게요🧋

In [ ]:
%%time
# train 데이터 불러오기
import pandas as pd
train = pd.read_feather("../input/riiid-train-data-multiple-formats/riiid_train.feather")
print("Train size:", train.shape)

제로백 20초 붕붕카를 탄 날아갈 것만 같은 이 가벼운 기분 😍

멋진 동료야~ 너무 고마워 👏👏👏

In [ ]:
%%time
# 남은 4개 데이터 불러오기
questions = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv')
lectures = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/lectures.csv')
example_test = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/example_test.csv')
example_sample_submission = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/example_sample_submission.csv')

# 2편 데이터 살펴보기

AI 교육 시간에 배웠던걸 써먹어 봐야지!

탐색적 데이터 분석이라고 했던가?? 

데이터를 처음 마주한 기분!! 너무 좋아 😃

In [ ]:
# 데이터 샘플
train.head()

In [ ]:
# 데이터 타입 및 메모리 확인
train.info()

캐글에서 주어지는 RAM용량은 12.4GB인데 train만 불러왔을 뿐인데 4Gb ㅠㅠ 

1/3을 벌써 써버리다니...

**멋진 동료**: 'object타입' 확인해보고 타입을 변경해보세요!! 그럼 용량이 줄어들걸요?

오~~ 오브젝트 타입??

In [ ]:
# prior_question_had_explanation 컬럼의 카테고리별 데이터 수
train['prior_question_had_explanation'].value_counts()

아 뭐야! True/False 밖에 없네!! 그럼 이걸 용량 작은 boolean 타입으로 변경해야지

In [ ]:
# prior_question_had_explanation 컬럼 타입 변경
train['prior_question_had_explanation'] = train['prior_question_had_explanation'].astype('boolean')
train.info()

좋아 좋아!! memory usage: 3.7+ GB 에서 memory usage: 3.1 GB 축소되었다

나머지 데이터 들도 대충 살펴보고 데이터 전처리 시작

# 3편 데이터 전처리

멋진 루돌프 프로덕트를 생각하며, 단디, 매매, 꼭꼭 데이터 전처리를 해보려고 하는데 

으악!!!계속해서"램 초과"메세지가 ㅠㅠ 

~~~
Your notebook tried to allocate more memory than is available. It has restarted
~~~

일이나 할 걸 그랬나 ㅎㅎ

어디선가 본 애자일 방식에 따르면 프로토타입이 중요하니 과감히 샘플링을!! 

이 모델 성공하면 대표님이 새로 나온 M2가 달린 메모리 24GB 정도 되는 빵빵한 장비 사주시겠지? (김칫국 벌컥) 기대기대 🤩

In [ ]:
# 앞뒤 돌아보지 않는 과감한 샘플링
print(train.shape)
train = train[:int(train.shape[0]/5)]
print(train.shape)

전처리 코드는 일단 고수님 코드 필사를... 공유해주셔서 감사합니다 :) 


https://www.kaggle.com/code/erikbruin/riiid-comprehensive-eda-baseline

In [ ]:
%%time

import numpy as np
#adding user features
user_df = train[train.answered_correctly != -1].groupby('user_id').agg({'answered_correctly': ['count', 'mean']}).reset_index()
user_df.columns = ['user_id', 'user_questions', 'user_mean']

user_lect = train.groupby(["user_id", "answered_correctly"]).size().unstack()
user_lect.columns = ['Lecture', 'Wrong', 'Right']
user_lect = user_lect[['Lecture']].fillna(0).astype('int8')
#user_lect = user_lect.astype('int8')
user_lect['watches_lecture'] = np.where(user_lect.Lecture > 0, 1, 0)
user_lect = user_lect.reset_index()

user_df = user_df.merge(user_lect, on = "user_id", how = "left")
del user_lect
user_df.head()

In [ ]:
cols = ['row_id', 'user_id', 'answered_correctly', 'content_id', 'prior_question_had_explanation', 'prior_question_elapsed_time']
train = train[cols]

In [ ]:
%%time
#adding content features
content_df = train[train.answered_correctly != -1].groupby('content_id').agg({'answered_correctly': ['count', 'mean']}).reset_index()
content_df.columns = ['content_id', 'content_questions', 'content_mean']
content_df.head(1)

In [ ]:
# 이렇게 나눠도 되려나??
from sklearn.model_selection import train_test_split

train = train.drop(columns="row_id")
train, validation = train_test_split(train, test_size=0.2, random_state=2022)

mean_prior = train.prior_question_elapsed_time.astype("float64").mean()

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_enc = LabelEncoder()

train = train.merge(user_df, on = "user_id", how = "left")
train = train.merge(content_df, on = "content_id", how = "left")
train['content_questions'].fillna(0, inplace = True)
train['content_mean'].fillna(0.5, inplace = True)
train['watches_lecture'].fillna(0, inplace = True)
train['user_questions'].fillna(0, inplace = True)
train['user_mean'].fillna(0.5, inplace = True)
train['prior_question_elapsed_time'].fillna(mean_prior, inplace = True)
train['prior_question_had_explanation'].fillna(False, inplace = True)
label_enc.fit(train['prior_question_had_explanation'])
train['prior_question_had_explanation'] = label_enc.transform(train['prior_question_had_explanation'])
train[['content_questions', 'user_questions']] = train[['content_questions', 'user_questions']].astype(int)
train.sample(5)

In [ ]:
validation = validation.merge(user_df, on = "user_id", how = "left")
validation = validation.merge(content_df, on = "content_id", how = "left")
validation['content_questions'].fillna(0, inplace = True)
validation['content_mean'].fillna(0.5, inplace = True)
validation['watches_lecture'].fillna(0, inplace = True)
validation['user_questions'].fillna(0, inplace = True)
validation['user_mean'].fillna(0.5, inplace = True)
validation['prior_question_elapsed_time'].fillna(mean_prior, inplace = True)
validation['prior_question_had_explanation'].fillna(False, inplace = True)
validation['prior_question_had_explanation'] = label_enc.transform(validation['prior_question_had_explanation'])
validation[['content_questions', 'user_questions']] = validation[['content_questions', 'user_questions']].astype(int)
validation.sample(5)

In [ ]:
features = ['user_questions', 'user_mean', 'content_questions', 'content_mean', 'prior_question_elapsed_time']

y_train = train['answered_correctly']
train = train[features]

y_val = validation['answered_correctly']
validation = validation[features]

복잡한 전처리는 끝냈고,

머신러닝 모델을 써야 하는데

루돌프는 200마력 정도면 썰매를 끌 수 있을 거야

기본이 100마력이니 200마력으로 업그레이드 (num_boost_round=200) 

부스터~ 빠라바라바라밤⚡️


### 머신러닝 모델

머신러닝 모델 중 성능 좋고 빠르다던 라잇찌비엠(lightGBM)으로 돌려 돌려!!

In [ ]:
params = {'objective': 'binary',
          'metric': 'auc',
          'seed': 2022,
         }

In [ ]:
import lightgbm as lgb
import gc

lgb_train = lgb.Dataset(train, y_train, categorical_feature = None)
lgb_eval = lgb.Dataset(validation, y_val, categorical_feature = None)
del train, y_train, validation, y_val
gc.collect()

In [ ]:
%%time
model = lgb.train(
    params, lgb_train,
    valid_sets=[lgb_train, lgb_eval],
    verbose_eval=10,
    num_boost_round=200,
    early_stopping_rounds=5
)

음하하하~ 돌아간다. (워닝은 일단 무시) 

이거면 투자도 받고 ai 교육을 혁신할 수 있을 것 같아 역사에 길이길이 남을 패러담임 변화를 도전해 보겠어!! 한 산업을 변화시킬거야! 하하하🔥

(멋진 동료에게 피드백 받는 중)

뭐라고? 서비스하기에는 성능이 부족하다고? 데이터에 대한 인사이트가 더 필요하다고? 

산타는 Knowledge Tracing 모델을 활용해 예측하고 이를 발전하기 위해...

놀리지 트래싱?

좋아! 그럼 난 놀리지 트래싱에 요즘 뜬다는 트랜스포머를 더해서 적용을??

여기서 말하는 트랜스포머는 변신 로봇 아님!! 머신러닝 모델(인코더와 디코더로 구성되고, 셀프 어텐선과... 블라블라~)

그 유명하다던  [Knowledge Tracing](https://medium.com/riiid-teamblog-kr/%EA%B5%90%EC%9C%A1ai%EC%9D%98-%EA%B8%B0%EB%B3%B8%EC%9D%B4%EC%9E%90-%EC%8B%9C%EC%9E%91-deep-knowledge-tracing-dkt-8bc132eda9ec)과 [Attention is All You Need](https://arxiv.org/abs/1706.03762) 논문을 읽는 중

좋은 거구나 그런데 무슨 말인지 모르겠다

외부에서 뛰어난 사람을 영입 해야지!

루돌프 데이터 전처리, 모델 파이프라인 설계, 모델 서빙해 줄 [머신러닝 엔지니어](https://riiid.com/en/career/machine-learning-engineer)를 모십니다.🍓

영입 될 때까진 내가 더 개선해봐야지!

오늘 하루 딴짓한 것을 무마하려고 채용링크라도 땋!!! 열 일하는 척하는 데브렐 매니저 입니다.

조금만 기다려줘요. 후편을 이어갈게요

*참고: 본 코드와 내용은 뤼이드 ai 조직과 전혀 관련 없는 코드 입니다.*